In [2]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
pip install split-folders

In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import split_folders
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt

from keras.models import model_from_json
import pickle
import math

Using TensorFlow backend.


In [ ]:
driveZip = '/content/drive/My Drive/temp/cropped_images.zip'

image_types = ["kurti","saree","none","shirt","tshirt"]
batch_size = 32
image_size = [224,224]
num_classes = 4

In [ ]:
import zipfile
with zipfile.ZipFile(driveZip,'r') as zip_dir:
    zip_dir.extractall(path='/content/')

In [7]:
split_folders.ratio('cropped_images', output="output", seed=1337, ratio= (.8,.2)) # default values

Copying files: 6383 files [00:01, 3231.91 files/s]


In [ ]:
# Image size should be [224,224]
# Iterate through each color folder

def resize_images(image_dir):
  for im_type in image_types:
    # Iterate through each image file in each image_type folder
    #  glob reads in any image with the extension "image_dir/im_type/*"
    for file in glob.glob(os.path.join(image_dir, im_type, "*")):
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((224,224), Image.ANTIALIAS)
        imResize.save(f + '.png', 'PNG', quality=90)

resize_images('/content/output/val')

In [9]:
!rm -r /content/output/train

!rm -r NOA
!mkdir NOA

!mkdir NOA/val
!mkdir NOA/val/none
!cp -r '/content/output/val/none/' '/content/NOA/val/'
!rm -r /content/output/val/none

rm: cannot remove 'NOA': No such file or directory


In [ ]:
!cp '/content/drive/My Drive/temp/VGG/secondLastModel.json' '/content/'
!cp '/content/drive/My Drive/temp/VGG/secondLastModel.h5' '/content/'

!cp '/content/drive/My Drive/temp/VGG/lastModel.json' '/content/'
!cp '/content/drive/My Drive/temp/VGG/lastModel.h5' '/content/'

!cp '/content/drive/My Drive/temp/VGG/logRModel.sav' '/content/'

In [11]:
json_file = open('secondLastModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
secondLastModel = model_from_json(loaded_model_json)

secondLastModel.load_weights("secondLastModel.h5")
secondLastModel.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
validation_set = val_datagen.flow_from_directory('/content/output/val/',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 1042 images belonging to 4 classes.


In [15]:
def getAllLabels(data_set):
    number_of_examples = len(data_set.filenames)
    number_of_generator_calls = math.ceil(number_of_examples / (1.0 * batch_size)) 
    # 1.0 above is to skip integer division
    test_labels = []
    for i in range(0,int(number_of_generator_calls)):
        test_labels.extend(np.array(data_set[i][1]))
    oneHotLabels = np.asarray(test_labels)
    labels = np.apply_along_axis(np.argmax,1, oneHotLabels)
    return labels

dtLabels = getAllLabels(validation_set)
print(dtLabels.shape)

(1042,)


In [ ]:
dtPredict = secondLastModel.predict_generator(validation_set,steps = len(validation_set))

In [17]:
print(dtPredict.shape)

(1042, 1000)


In [18]:
labels = np.full(dtPredict.shape[0],1)
test1Set = np.concatenate((dtPredict,np.asarray([labels]).T), axis=1)
print(test1Set.shape)

(1042, 1001)


In [19]:
validation_set = val_datagen.flow_from_directory('/content/NOA/val/',
                                            target_size = (224, 224),
                                            batch_size = batch_size)

Found 236 images belonging to 1 classes.


In [ ]:
predict = secondLastModel.predict_generator(validation_set,steps = len(validation_set))

In [21]:
labels = np.full(predict.shape[0],0,np.float32)
test0Set = np.concatenate((predict,np.asarray([labels]).T), axis=1)
print(test0Set.shape)

(236, 1001)


In [ ]:
def predictNOA(testData):
    testX = testData[:,:-1]
    testY = testData[:,-1]
    print(testX.shape)
    print(testY.shape)

    clf = pickle.load(open('logRModel.sav', 'rb'))
    out = clf.predict(testX)
    return out

def getCount(x,val):
    return np.where(x == val)[0].shape[0]

def getCorectSamples(dataset,out,correctVal):
    newSet = []
    correctInds = np.where(out == correctVal)[0]
    for ind in correctInds:
        newSet.append(dataset[ind])
    return np.asarray(newSet)

def wrongLabelsCount(x,y):
    return np.where(x != y)[0].shape[0]

In [ ]:
totalSamples = test0Set.shape[0] + test1Set.shape[0]
wrongSamples = 0

In [24]:
out = predictNOA(test0Set)
wrongSamples += getCount(out,1)

(236, 1000)
(236,)


In [25]:
out = predictNOA(test1Set)
wrongSamples += getCount(out,0)

(1042, 1000)
(1042,)


In [26]:
newSet = getCorectSamples(test1Set,out,1)
print(newSet.shape)

(961, 1001)


In [27]:
json_file = open('lastModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
lastModel = model_from_json(loaded_model_json)

lastModel.load_weights("lastModel.h5")
lastModel.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 4004      
Total params: 4,004
Trainable params: 4,004
Non-trainable params: 0
_________________________________________________________________


In [ ]:
softMaxLabs = lastModel.predict(dtPredict)
predLabs = np.apply_along_axis(np.argmax,1, softMaxLabs)

In [29]:
print(predLabs.shape,dtLabels.shape)

(1042,) (1042,)


In [ ]:
wrongSamples += wrongLabelsCount(predLabs,dtLabels)

In [31]:
print('Accuracy: ',(totalSamples - wrongSamples)/totalSamples)

Accuracy:  0.7816901408450704
